In [5]:
import numpy as np
import matplotlib.pyplot as plt
from pca import PCA
from knn import KNNClassifier

In [19]:
def read_mnist_images(file_path):
    with open(file_path, 'rb') as f:
        magic_number = int.from_bytes(f.read(4), 'big')
        num_images = int.from_bytes(f.read(4), 'big')
        num_rows = int.from_bytes(f.read(4), 'big')
        num_cols = int.from_bytes(f.read(4), 'big')

        image_data = np.frombuffer(f.read(), dtype=np.uint8)

    # Reshape to (num_images, num_rows, num_cols)
    images = image_data.reshape(num_images, num_rows, num_cols)
    
    # Normalize
    images = images / 255.0
    
    # Flatten the images (called feature matrix)
    flattened_images = images.reshape(num_images, -1)

    return flattened_images


def read_mnist_labels(file_path):
    with open(file_path, 'rb') as f:
        magic_number = int.from_bytes(f.read(4), 'big')
        num_labels = int.from_bytes(f.read(4), 'big')

        label_data = np.frombuffer(f.read(), dtype=np.uint8)

    return label_data

In [20]:
train_images = read_mnist_images('data/train-images-idx3-ubyte')
train_labels = read_mnist_labels('data/train-labels-idx1-ubyte')

test_images = read_mnist_images('data/t10k-images-idx3-ubyte')
test_labels = read_mnist_labels('data/t10k-labels-idx1-ubyte')

print(f"Training images shape: {train_images.shape}")
print(f"Training labels shape: {train_labels.shape}")
print(f"Test images shape: {test_images.shape}")
print(f"Test labels shape: {test_labels.shape}")

Training images shape: (60000, 784)
Training labels shape: (60000,)
Test images shape: (10000, 784)
Test labels shape: (10000,)
